In [33]:
##importamos librerias
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, PHONE_NUMBER, API_KEY_WAPI

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

import pandas as pd
import requests
from tqdm import tqdm 


## REQUEST

In [34]:
## Construimos el url para hacer nuestra solicitud
query = 'Medellin'
api_key = API_KEY_WAPI

url_clima = "http://api.weatherapi.com/v1/forecast.json?key="+api_key+"&q="+query+"&days=1&aqi=no&alerts=no"

response = requests.get(url_clima).json()

In [35]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [36]:
response['forecast']['forecastday'][0].keys()


dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [37]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[0] #Fecha
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(":")[0]) #hora


1

In [38]:
response['forecast']['forecastday'][0]['hour'][1]['condition']['text']

'Fog'

In [39]:
response['forecast']['forecastday'][0]['hour'][0]['temp_c'] #temperatura
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain'] #Rain probability
response['forecast']['forecastday'][0]['hour'][0]['chance_of_rain'] #Rain probability

65

## DATAFRAME

In [40]:
def get_forecast(response, i): 
    
    date = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0] #Fecha
    hour = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(":")[0])
    condition = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    temp = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return date, hour, condition, temp, rain, prob_rain
    
    

In [41]:
weather_data = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    weather_data.append(get_forecast(response, i)) 
    


100%|██████████| 24/24 [00:00<00:00, 53261.00it/s]


In [42]:
weather_data


[('2023-06-01', 0, 'Light rain shower', 13.1, 0, 65),
 ('2023-06-01', 1, 'Fog', 13.3, 0, 0),
 ('2023-06-01', 2, 'Fog', 13.1, 1, 80),
 ('2023-06-01', 3, 'Fog', 13.4, 0, 0),
 ('2023-06-01', 4, 'Light rain', 13.7, 1, 84),
 ('2023-06-01', 5, 'Fog', 13.5, 1, 71),
 ('2023-06-01', 6, 'Light rain shower', 13.2, 1, 87),
 ('2023-06-01', 7, 'Fog', 13.9, 0, 0),
 ('2023-06-01', 8, 'Cloudy', 15.0, 0, 0),
 ('2023-06-01', 9, 'Partly cloudy', 15.7, 0, 0),
 ('2023-06-01', 10, 'Light rain shower', 15.7, 1, 75),
 ('2023-06-01', 11, 'Partly cloudy', 17.3, 0, 0),
 ('2023-06-01', 12, 'Light rain shower', 18.6, 1, 78),
 ('2023-06-01', 13, 'Light rain shower', 18.3, 1, 87),
 ('2023-06-01', 14, 'Light rain shower', 17.8, 0, 66),
 ('2023-06-01', 15, 'Mist', 17.3, 0, 0),
 ('2023-06-01', 16, 'Light rain shower', 17.3, 1, 74),
 ('2023-06-01', 17, 'Cloudy', 17.0, 0, 0),
 ('2023-06-01', 18, 'Light rain shower', 15.8, 1, 74),
 ('2023-06-01', 19, 'Fog', 15.1, 0, 0),
 ('2023-06-01', 20, 'Light rain shower', 14.5, 0, 70)

In [43]:
col = ['Date', 'Hour', 'Condition', 'Temp', 'Rain', '"%" of Rain']
weather_df = pd.DataFrame(weather_data, columns=col)
weather_df


,Date,Hour,Condition,Temp,Rain,"""%"" of Rain"
0,2023-06-01,0,Light rain shower,13.1,0,65
1,2023-06-01,1,Fog,13.3,0,0
2,2023-06-01,2,Fog,13.1,1,80
3,2023-06-01,3,Fog,13.4,0,0
4,2023-06-01,4,Light rain,13.7,1,84
5,2023-06-01,5,Fog,13.5,1,71
6,2023-06-01,6,Light rain shower,13.2,1,87
7,2023-06-01,7,Fog,13.9,0,0
8,2023-06-01,8,Cloudy,15.0,0,0
9,2023-06-01,9,Partly cloudy,15.7,0,0


In [44]:
rain_df = weather_df[(weather_df["Rain"]==1) & (weather_df["Hour"]<22) & (weather_df["Hour"]>5)]
rain_df = rain_df[['Hour','Condition']]
rain_df.set_index('Hour', inplace=True)
rain_df

,Condition
Hour,
6,Light rain shower
10,Light rain shower
12,Light rain shower
13,Light rain shower
16,Light rain shower
18,Light rain shower


In [45]:
weather_message = f'\n Hellow! \n\n\n The weather forecast for {weather_df["Date"][1]} in Medellín: \n\n\n {rain_df}'
weather_message

'\n Hellow! \n\n\n The weather forecast for 2023-06-01 in Medellín: \n\n\n               Condition\nHour                   \n6     Light rain shower\n10    Light rain shower\n12    Light rain shower\n13    Light rain shower\n16    Light rain shower\n18    Light rain shower'

In [46]:
PHONE_NUMBER

'+1 361 310 4693'

In [47]:
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN
client = Client(account_sid, auth_token)

message = client.messages \
    .create(
         body=weather_message,
         from_=PHONE_NUMBER,
         to='+573145103005'
     )

print('Mensaje enviado', message.sid)

Mensaje enviado SM861ed403cdee0dcf32037bc2936d70b3


In [49]:
message.status

'queued'